In [1]:
%matplotlib inline
%load_ext autoreload
%autoreload 2

import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
from IPython.display import Markdown, display, HTML
from collections import defaultdict

import torch
import torch.nn as nn
import torch.optim as optim
from livelossplot import PlotLosses

# Fix the dying kernel problem (only a problem in some installations - you can remove it, if it works without it)
import os
os.environ['KMP_DUPLICATE_LIB_OK'] = 'True'

# Load the dataset for recommenders

In [2]:
data_path = os.path.join("data", "hotel_data")

interactions_df = pd.read_csv(os.path.join(data_path, "hotel_data_interactions_df.csv"), index_col=0)

base_item_features = ['term', 'length_of_stay_bucket', 'rate_plan', 'room_segment', 'n_people_bucket', 'weekend_stay']

column_values_dict = {
    'term': ['WinterVacation', 'Easter', 'OffSeason', 'HighSeason', 'LowSeason', 'MayLongWeekend', 'NewYear', 'Christmas'],
    'length_of_stay_bucket': ['[0-1]', '[2-3]', '[4-7]', '[8-inf]'],
    'rate_plan': ['Standard', 'Nonref'],
    'room_segment': ['[0-160]', '[160-260]', '[260-360]', '[360-500]', '[500-900]'],
    'n_people_bucket': ['[1-1]', '[2-2]', '[3-4]', '[5-inf]'],
    'weekend_stay': ['True', 'False']
}

interactions_df.loc[:, 'term'] = pd.Categorical(
    interactions_df['term'], categories=column_values_dict['term'])
interactions_df.loc[:, 'length_of_stay_bucket'] = pd.Categorical(
    interactions_df['length_of_stay_bucket'], categories=column_values_dict['length_of_stay_bucket'])
interactions_df.loc[:, 'rate_plan'] = pd.Categorical(
    interactions_df['rate_plan'], categories=column_values_dict['rate_plan'])
interactions_df.loc[:, 'room_segment'] = pd.Categorical(
    interactions_df['room_segment'], categories=column_values_dict['room_segment'])
interactions_df.loc[:, 'n_people_bucket'] = pd.Categorical(
    interactions_df['n_people_bucket'], categories=column_values_dict['n_people_bucket'])
interactions_df.loc[:, 'weekend_stay'] = interactions_df['weekend_stay'].astype('str')
interactions_df.loc[:, 'weekend_stay'] = pd.Categorical(
    interactions_df['weekend_stay'], categories=column_values_dict['weekend_stay'])

n_users = np.max(interactions_df['user_id']) + 1
n_items = np.max(interactions_df['item_id']) + 1

display(HTML(interactions_df.head(15).to_html()))

,user_id,item_id,term,length_of_stay_bucket,rate_plan,room_segment,n_people_bucket,weekend_stay
0,1,0,WinterVacation,[2-3],Standard,[260-360],[5-inf],True
1,2,1,WinterVacation,[2-3],Standard,[160-260],[3-4],True
2,3,2,WinterVacation,[2-3],Standard,[160-260],[2-2],False
3,4,3,WinterVacation,[4-7],Standard,[160-260],[3-4],True
4,5,4,WinterVacation,[4-7],Standard,[0-160],[2-2],True
5,6,5,Easter,[4-7],Standard,[260-360],[5-inf],True
6,7,6,OffSeason,[2-3],Standard,[260-360],[5-inf],True
7,8,7,HighSeason,[2-3],Standard,[160-260],[1-1],True
8,9,8,HighSeason,[2-3],Standard,[0-160],[1-1],True
9,8,7,HighSeason,[2-3],Standard,[160-260],[1-1],True


# (Optional) Prepare numerical user features

The method below is left here for convenience if you want to experiment with content-based user features as an input for your neural network.

In [3]:
def prepare_users_df(interactions_df):
    
    # Make copy of interactions_df
    users_df = interactions_df.copy()
        
    # Get dummies
    users_df = pd.get_dummies(users_df, columns=[
        'term',
        'length_of_stay_bucket',
        'rate_plan',
        'room_segment',
        'n_people_bucket',
        'weekend_stay'
    ])
    
    # Drop column item_id
    users_df = users_df.drop(columns=['item_id'])
    
    # Group data by user_id using sum
    users_df = users_df.groupby('user_id').sum()
    
    # Dividing the numerical data in order to obtain the probability distribution of features for a given user
    users_df = users_df.div(users_df.sum(axis=1) / 6, axis=0)
    
    # Add prefix to columns
    users_df = users_df.add_prefix('user_')
    
    # Reset index
    users_df = users_df.reset_index()
    
    # Get list of user features
    user_features = users_df.columns.tolist()[1:]
    
    return users_df, user_features
    
    
users_df, user_features = prepare_users_df(interactions_df)


display(user_features)
display(users_df.loc[users_df['user_id'].isin([706, 1736, 7779, 96, 1, 50, 115])].head(15))

['user_term_Christmas',
 'user_term_Easter',
 'user_term_HighSeason',
 'user_term_LowSeason',
 'user_term_MayLongWeekend',
 'user_term_NewYear',
 'user_term_OffSeason',
 'user_term_WinterVacation',
 'user_length_of_stay_bucket_[0-1]',
 'user_length_of_stay_bucket_[2-3]',
 'user_length_of_stay_bucket_[4-7]',
 'user_length_of_stay_bucket_[8-inf]',
 'user_rate_plan_Nonref',
 'user_rate_plan_Standard',
 'user_room_segment_[0-160]',
 'user_room_segment_[160-260]',
 'user_room_segment_[260-360]',
 'user_room_segment_[360-500]',
 'user_n_people_bucket_[1-1]',
 'user_n_people_bucket_[2-2]',
 'user_n_people_bucket_[3-4]',
 'user_n_people_bucket_[5-inf]',
 'user_weekend_stay_False',
 'user_weekend_stay_True']

,user_id,user_term_Christmas,user_term_Easter,user_term_HighSeason,user_term_LowSeason,user_term_MayLongWeekend,user_term_NewYear,user_term_OffSeason,user_term_WinterVacation,user_length_of_stay_bucket_[0-1],...,user_room_segment_[0-160],user_room_segment_[160-260],user_room_segment_[260-360],user_room_segment_[360-500],user_n_people_bucket_[1-1],user_n_people_bucket_[2-2],user_n_people_bucket_[3-4],user_n_people_bucket_[5-inf],user_weekend_stay_False,user_weekend_stay_True
0,1,0.0,0.0,0.090909,0.136364,0.000000,0.000000,0.681818,0.090909,0.000000,...,0.000000,0.863636,0.136364,0.0,0.000000,0.727273,0.181818,0.090909,0.227273,0.772727
40,50,0.0,0.0,0.304348,0.217391,0.000000,0.000000,0.434783,0.043478,0.000000,...,0.000000,0.565217,0.434783,0.0,0.000000,0.173913,0.521739,0.304348,0.217391,0.782609
84,96,0.0,0.0,0.136364,0.045455,0.045455,0.000000,0.681818,0.090909,0.272727,...,0.045455,0.863636,0.090909,0.0,0.045455,0.272727,0.590909,0.090909,0.272727,0.727273
102,115,0.0,0.0,0.000000,0.000000,0.000000,0.000000,1.000000,0.000000,0.000000,...,0.000000,0.500000,0.500000,0.0,0.500000,0.000000,0.000000,0.500000,1.000000,0.000000
371,706,0.0,0.0,0.190855,0.143141,0.047714,0.011928,0.512922,0.095427,0.298211,...,0.035785,0.858847,0.107356,0.0,0.119284,0.155070,0.584493,0.131213,0.417495,0.584493
1383,1736,0.0,0.0,0.206897,0.275862,0.000000,0.000000,0.482759,0.034483,0.241379,...,0.000000,0.931034,0.068966,0.0,0.379310,0.413793,0.206897,0.000000,0.551724,0.448276
7301,7779,0.0,0.0,0.000000,0.500000,0.000000,0.000000,0.500000,0.000000,0.000000,...,0.000000,1.000000,0.000000,0.0,0.000000,0.000000,0.750000,0.250000,0.250000,0.750000


# (Optional) Prepare numerical item features

The method below is left here for convenience if you want to experiment with content-based item features as an input for your neural network.

In [4]:
def prepare_items_df(interactions_df):
    
    # Create copy of DataFrame
    items_df = interactions_df.copy()
    
    # Drop column user_id if present and drop duplicates
    if 'user_id' in items_df.columns:
        items_df = items_df.drop(columns=['user_id'])
        items_df = items_df.drop_duplicates()
        
    
    # Get dummies
    items_df = pd.get_dummies(items_df, columns=[
        'term',
        'length_of_stay_bucket',
        'rate_plan',
        'room_segment',
        'n_people_bucket',
        'weekend_stay'
    ], dtype=float)
    
    # Get list of item features
    item_features = items_df.columns.tolist()[1:]
    
    return items_df, item_features

items_df, item_features = prepare_items_df(interactions_df)

display(item_features)

display(items_df.loc[items_df['item_id'].isin([0, 1, 2, 3, 4, 5, 6])].head(15))

['term_Christmas',
 'term_Easter',
 'term_HighSeason',
 'term_LowSeason',
 'term_MayLongWeekend',
 'term_NewYear',
 'term_OffSeason',
 'term_WinterVacation',
 'length_of_stay_bucket_[0-1]',
 'length_of_stay_bucket_[2-3]',
 'length_of_stay_bucket_[4-7]',
 'length_of_stay_bucket_[8-inf]',
 'rate_plan_Nonref',
 'rate_plan_Standard',
 'room_segment_[0-160]',
 'room_segment_[160-260]',
 'room_segment_[260-360]',
 'room_segment_[360-500]',
 'n_people_bucket_[1-1]',
 'n_people_bucket_[2-2]',
 'n_people_bucket_[3-4]',
 'n_people_bucket_[5-inf]',
 'weekend_stay_False',
 'weekend_stay_True']

,item_id,term_Christmas,term_Easter,term_HighSeason,term_LowSeason,term_MayLongWeekend,term_NewYear,term_OffSeason,term_WinterVacation,length_of_stay_bucket_[0-1],...,room_segment_[0-160],room_segment_[160-260],room_segment_[260-360],room_segment_[360-500],n_people_bucket_[1-1],n_people_bucket_[2-2],n_people_bucket_[3-4],n_people_bucket_[5-inf],weekend_stay_False,weekend_stay_True
0,0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,...,0.0,0.0,1.0,0.0,0.0,0.0,0.0,1.0,0.0,1.0
1,1,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,...,0.0,1.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,1.0
2,2,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,...,0.0,1.0,0.0,0.0,0.0,1.0,0.0,0.0,1.0,0.0
3,3,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,...,0.0,1.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,1.0
4,4,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,...,1.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,1.0
5,5,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,1.0,0.0,0.0,0.0,0.0,1.0,0.0,1.0
6,6,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,...,0.0,0.0,1.0,0.0,0.0,0.0,0.0,1.0,0.0,1.0


# Negative interactions

In [5]:
def generateNegativeInteractions(interactions_df, seed=6789, n_neg_per_pos=5):
    
    # Set random seed
    rng = np.random.RandomState(seed=seed)
    
    # Get number of users and items
    n_users = np.max(interactions_df['user_id']) + 1
    n_items = np.max(interactions_df['item_id']) + 1
    
    # Generate interaction matrix
    r = np.zeros(shape=(n_users, n_items))
    for idx, interaction in interactions_df.iterrows():
        r[int(interaction['user_id'])][int(interaction['item_id'])] = 1
    
    # Generate negative interactions
    
    negative_interactions = []
    
    i = 0
    while i < n_neg_per_pos * len(interactions_df):
        sample_size = 1000
        user_ids = rng.choice(np.arange(n_users), size=sample_size)
        item_ids = rng.choice(np.arange(n_items), size=sample_size)

        j = 0
        while j < sample_size and i < n_neg_per_pos * len(interactions_df):
            if r[user_ids[j]][item_ids[j]] == 0:
                negative_interactions.append([user_ids[j], item_ids[j], 0])
                i += 1
            j += 1
    
    return negative_interactions

negative_interactions = generateNegativeInteractions(interactions_df)
display(negative_interactions[:50])

[[10077, 10, 0],
 [2847, 84, 0],
 [14316, 64, 0],
 [9409, 234, 0],
 [14080, 405, 0],
 [966, 646, 0],
 [13510, 505, 0],
 [6813, 128, 0],
 [802, 234, 0],
 [10612, 541, 0],
 [1351, 351, 0],
 [4846, 600, 0],
 [1339, 404, 0],
 [6464, 243, 0],
 [8736, 35, 0],
 [4277, 728, 0],
 [254, 60, 0],
 [3305, 349, 0],
 [10420, 631, 0],
 [3500, 554, 0],
 [8539, 125, 0],
 [13197, 38, 0],
 [7780, 435, 0],
 [8452, 196, 0],
 [2033, 493, 0],
 [12778, 17, 0],
 [2661, 231, 0],
 [1907, 670, 0],
 [13433, 87, 0],
 [13392, 8, 0],
 [5554, 427, 0],
 [6213, 111, 0],
 [1732, 718, 0],
 [13390, 26, 0],
 [1179, 613, 0],
 [5480, 614, 0],
 [8000, 581, 0],
 [13243, 349, 0],
 [8059, 706, 0],
 [12434, 259, 0],
 [8056, 362, 0],
 [6882, 415, 0],
 [1372, 715, 0],
 [6134, 245, 0],
 [4150, 735, 0],
 [188, 124, 0],
 [11092, 157, 0],
 [3649, 439, 0],
 [5639, 580, 0],
 [9231, 497, 0]]

# Neural network recommender

<span style="color:red"><font size="4">**Task:**</font></span><br> 
Code a recommender based on a neural network model. You are free to choose any network architecture you find appropriate. The network can use the interaction vectors for users and items, embeddings of users and items, as well as user and item features (you can use the features you developed in the first project).

Remember to keep control over randomness - in the init method add the seed as a parameter and initialize the random seed generator with that seed (both for numpy and pytorch):

```python
self.seed = seed
self.rng = np.random.RandomState(seed=seed)
```
in the network model:
```python
self.seed = torch.manual_seed(seed)
```

You are encouraged to experiment with:
  - the number of layers in the network, the number of neurons and different activation functions,
  - different optimizers and their parameters,
  - batch size and the number of epochs,
  - embedding layers,
  - content-based features of both users and items.

In [6]:
from recommenders.recommender import Recommender


class NNRecommender(Recommender):
    """
    Recommender class.
    """
    
    def __init__(self, seed=6789, n_neg_per_pos=5, **params):
        """
        Initialize recommender params and variables.
        """
        self.model = None
        self.optimizer = None
        self.n_neg_per_pos = n_neg_per_pos
        
        self.recommender_df = pd.DataFrame(columns=['user_id', 'item_id', 'score'])
        self.users_df = None
        self.user_features = None
        
        self.seed = seed
        self.rng = np.random.RandomState(seed=seed)
        
        # Set device for GPU if available
        if 'device' in params:
            self.device = params['device']
        else:
            self.device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
            
        # Learning rate
        if 'lr' in params:  # learning rate
            self.lr = params['lr']
        else:
            self.lr = 0.01
        
        # L2 regularization weight
        if 'weight_decay' in params:
            self.weight_decay = params['weight_decay']
        else:
            self.weight_decay = 0.001
        
        # Number of epochs (each epoch goes through the entire training set)
        if 'n_epochs' in params:
            self.n_epochs = int(params['n_epochs'])
        else:
            self.n_epochs = 30
            
        # Batch size
        if 'batch_size' in params:
            self.batch_size = params['batch_size']
        else:
            self.batch_size = 128
        
        # Item and User Embedding size
        if 'embedding_dim' in params:
            self.embedding_dim = params['embedding_dim']
        else:
            self.embedding_dim = 16
          
        # Parameter if use content-based features (for hybrid recommender)
        if 'hybrid' in params:
            self.hybrid = params['hybrid']
        else:
            self.hybrid = False
            
        # Parameter to specify model architecture
        if 'model_id' in params:
            self.model_id = params['model_id']
        else:
            self.model_id = 1
            
    def fit(self, interactions_df, users_df, items_df):
        """
        Training of the recommender.
        
        :param pd.DataFrame interactions_df: DataFrame with recorded interactions between users and items 
            defined by user_id, item_id and features of the interaction.
        :param pd.DataFrame users_df: DataFrame with users and their features defined by user_id and the user feature columns.
        :param pd.DataFrame items_df: DataFrame with items and their features defined by item_id and the item feature columns.
        """
        
        # Use global variables for number of items and users in interactions_df
        global n_items, n_users
        
        # Create copy of interactions_df
        interactions_df = interactions_df.copy()
        
        # --- Prepare users_df and items_df (users and items features) --- #
        if self.hybrid:
            # Prepare users content-based features
            users_df, user_features = prepare_users_df(interactions_df)
            
            self.users_df = users_df
            self.user_features = user_features

            # Prepare items content-based features
            items_df, item_features = prepare_items_df(interactions_df)
            items_df = items_df.loc[:, ['item_id'] + item_features]
        
        # Set up interated columns for positive interations
        interactions_df.loc[:, 'interacted'] = 1
        
        # Generate negative interactions
        negative_interactions = generateNegativeInteractions(interactions_df, seed=self.seed, n_neg_per_pos=self.n_neg_per_pos)
    
        # Concat negative interactions with positive interations
        interactions_df = pd.concat(
            [interactions_df, pd.DataFrame(negative_interactions, columns=['user_id', 'item_id', 'interacted'])]).reset_index(drop=True)
        
        # Merge interactions with users and items content-based features
        if self.hybrid:
            interactions_df = pd.merge(interactions_df, users_df, on=['user_id'])
            interactions_df = pd.merge(interactions_df, items_df, on=['item_id'])

            dot_cols = ["dot_" + col for col in item_features]
              
            # Item and user features dot product
            interactions_df[dot_cols] = interactions_df[user_features] \
                * interactions_df[item_features].values

        # Initialize the neural network model
        self.model = NeuralNetwork(
            n_items=n_items,
            n_users=n_users,
            embedding_dim=self.embedding_dim, 
            seed=self.seed, 
            hybrid=self.hybrid,
            model_id=self.model_id
        )
        
        # Send model to GPU (if available)
        self.model.to(self.device)
        
        # Initialize optimizer for neural network parameters with learning rate and L2 regularization
        self.optimizer = optim.Adam(self.model.parameters(), lr=self.lr, weight_decay=self.weight_decay)
        
        # --- Train the model using an optimizer --- #
        
        # Permutate training dataset items
        interaction_ids = self.rng.permutation(len(interactions_df))
        training_ids = interaction_ids
        
        # Iterate over whole training dataset for epochs
        for epoch in range(self.n_epochs):
            
            # Shuffle training dataset items
            self.rng.shuffle(training_ids)
            
            # Calculate number of batches based on batch_size parameter
            batch_idx = 0
            n_batches = int(np.ceil(len(interaction_ids) / self.batch_size))
            
            # Iterate over batches
            for batch_idx in range(n_batches):
                
                # Get batch elements
                batch_ids = training_ids[(batch_idx * self.batch_size):((batch_idx + 1) * self.batch_size)]
                batch = interactions_df.loc[batch_ids]
                
                if self.hybrid:
                    columns = ['user_id', 'item_id'] + dot_cols + user_features + item_features
                else:
                    columns = ['user_id', 'item_id']
                
                # Convert batch to tensor and send it to GPU (if available)
                batch_input = torch.from_numpy(batch.loc[:, columns].to_numpy()).long().to(self.device)
                
                # Convert target (interacted column value) to tensor and send it to GPU (if available)
                y_target = torch.from_numpy(batch.loc[:, ['interacted']].values).float().to(self.device)
                
                # Forward Pass
                y = self.model(batch_input).clip(0.000001, 0.999999)
                
                # --- Define loss function and backpropagate --- #
                
                # Clear optimizer gradients
                self.optimizer.zero_grad()
                
                # Loss function
                loss = -(y_target * y.log() + (1 - y_target) * (1 - y).log()).sum()
                
                # Backpropagate (Backward Pass)
                loss.backward()
                
                # Optimizer step (update parameters based on gradients)
                self.optimizer.step()
    
    def recommend(self, users_df, items_df, n_recommendations=1):
        """
        Serving of recommendations. Scores items in items_df for each user in users_df and returns 
        top n_recommendations for each user.
        
        :param pd.DataFrame users_df: DataFrame with users and their features for which recommendations should be generated.
        :param pd.DataFrame items_df: DataFrame with items and their features which should be scored.
        :param int n_recommendations: Number of recommendations to be returned for each user.
        :return: DataFrame with user_id, item_id and score as columns returning n_recommendations top recommendations 
            for each user.
        :rtype: pd.DataFrame
        """
        
        # Clean previous recommendations (iloc could be used alternatively)
        self.recommender_df = self.recommender_df[:0]
        
        # Prepare users_df and items_df
        if self.hybrid:
            users = users_df.loc[:, 'user_id']
            features_data = pd.merge(users, self.users_df, on=['user_id'], how='left').fillna(0)

            items_df, item_features = prepare_items_df(items_df)
            items_df = items_df.loc[:, ['item_id'] + item_features]
            
            # Cross product all users with all items
            features_data = pd.merge(features_data, items_df, how='cross')
            
            dot_cols = ["dot_" + col for col in item_features]
            
            # Item and user features dot product
            features_data[dot_cols] = features_data[self.user_features] \
                * features_data[item_features].values
        
        # Score the items
        recommendations = pd.DataFrame(columns=['user_id', 'item_id', 'score'])
        
        for ix, user in users_df.iterrows():
            
            user_id = user['user_id']
            items_ids = items_df['item_id'].tolist()
            
            # --- Calculate the score for the user and every item in items_df --- #
            if not self.hybrid:
                nn_input = torch.tensor(list(zip([user_id] * len(items_ids), items_ids))).to(self.device)
            else:
                columns = ['user_id', 'item_id'] + dot_cols + self.user_features + item_features
                data = features_data.loc[(features_data['user_id'] == user_id), columns]
                nn_input = torch.tensor(data.values).long().to(self.device)
            
            # Get items scores
            scores = self.model(nn_input).flatten().detach().cpu().numpy()

            # Sort items scores and select best n results
            chosen_ids = np.argsort(-scores)[:n_recommendations]
            
            # Prepare recommendations
            recommendations = []
            for item_id in chosen_ids:
                recommendations.append(
                    {
                        'user_id': user['user_id'],
                        'item_id': item_id,
                        'score': scores[item_id]
                    }
                )
            
            user_recommendations = pd.DataFrame(recommendations)

            self.recommender_df = pd.concat([self.recommender_df, user_recommendations])

        return self.recommender_df

    
# Neural Network Models
class NeuralNetwork(nn.Module):
    def __init__(self, n_items, n_users, embedding_dim, seed, model_id=1, hybrid=False):
        super().__init__()

        # Set PyTorch seed
        self.seed = torch.manual_seed(seed)
        
        # Set model_id and hybrid
        self.model_id = model_id
        self.hybrid = hybrid
        
        # Item embedding layer
        self.item_embedding = nn.Embedding(n_items, embedding_dim)
        
        # User embedding layer
        self.user_embedding = nn.Embedding(n_users, embedding_dim)
        
        # --- Different architectures --- #
        
        # * User + Item Embedding Models * #
        if not hybrid:
            
            # [1] One fully-connected layer without bias
            if model_id == 1:
                self.fc = nn.Linear(embedding_dim, 1, bias=False)

            # [2] Two fully-connected layers without bias
            if model_id == 2:
                self.fc1 = nn.Linear(embedding_dim, 8, bias=False)
                self.fc2 = nn.Linear(8, 1, bias=False)

            # [3] Three fully-connected layers without bias
            if model_id == 3:
                self.fc1 = nn.Linear(embedding_dim, 16, bias=False)
                self.fc2 = nn.Linear(16, 8, bias=False)
                self.fc3 = nn.Linear(8, 1, bias=False)

            # [4] Four fully-connected layers without bias
            if model_id == 4:
                self.fc1 = nn.Linear(embedding_dim, 32, bias=False)
                self.fc2 = nn.Linear(32, 16, bias=False)
                self.fc3 = nn.Linear(16, 8, bias=False)
                self.fc4 = nn.Linear(8, 1, bias=False)

            # [5] Five fully-connected layers with bias
            if model_id == 5:
                self.fc1 = nn.Linear(embedding_dim, 256)
                self.fc2 = nn.Linear(256, 128)
                self.fc3 = nn.Linear(128, 64)
                self.fc4 = nn.Linear(64, 32)
                self.fc5 = nn.Linear(32, 1)

            # [6] Six fully-connected layers with bias
            if model_id == 6:
                self.fc1 = nn.Linear(embedding_dim, 256)
                self.fc2 = nn.Linear(256, 128)
                self.fc3 = nn.Linear(128, 64)
                self.fc4 = nn.Linear(64, 32)
                self.fc5 = nn.Linear(32, 16)
                self.fc6 = nn.Linear(16, 1)

            # [7] GMF + MLP
            if model_id == 7:
                # GMF
                self.gmf_user_embedding = nn.Embedding(n_users, embedding_dim)
                self.gmf_item_embedding = nn.Embedding(n_items, embedding_dim)

                # MLP
                self.mlp_user_embedding = nn.Embedding(n_users, embedding_dim)
                self.mlp_item_embedding = nn.Embedding(n_items, embedding_dim)
                self.mlp_fc1 = nn.Linear(2 * embedding_dim, 16)
                self.mlp_fc2 = nn.Linear(16, 8)

                # Merge
                self.fc = nn.Linear(embedding_dim + 8, 1, bias=False)
        
        # * Hybrid Models * #
        if hybrid:
            
            # [1] One fully-connected layer without bias
            if model_id == 1:
                self.fc = nn.Linear(embedding_dim + 24, 1, bias=False)
                
            # [2] Three fully-connected layer with bias
            if model_id == 2:
                self.embedding = nn.Linear(embedding_dim, 4)
                
                self.fc_content_1 = nn.Linear(24, 64)
                self.fc_content_2 = nn.Linear(64, 32)
                self.fc_content_3 = nn.Linear(32, 8)
                
                self.fc = nn.Linear(12, 1)
                
            # [3] GMF + MLP + Content-based #1
            if model_id == 3:
                # GMF
                self.gmf_user_embedding = nn.Embedding(n_users, embedding_dim)
                self.gmf_item_embedding = nn.Embedding(n_items, embedding_dim)

                # MLP
                self.mlp_user_embedding = nn.Embedding(n_users, embedding_dim)
                self.mlp_item_embedding = nn.Embedding(n_items, embedding_dim)
                self.mlp_fc1 = nn.Linear(2 * embedding_dim, 16)
                self.mlp_fc2 = nn.Linear(16, 8)
                
                # Content-based (dot product)
                self.content_based = nn.Linear(24, 8)
                
                # Content-based (separately)
                self.cb_fc1 = nn.Linear(2 * 24, 16)
                self.cb_fc2 = nn.Linear(16, 8)
                
                # Merge
                self.fc1 = nn.Linear(embedding_dim + 8 + 8 + 8, 16)
                self.fc2 = nn.Linear(16, 1)
                
            # [4] GMF + MLP + Content-based #2
            if model_id == 4:
                # GMF
                self.gmf_user_embedding = nn.Embedding(n_users, embedding_dim)
                self.gmf_item_embedding = nn.Embedding(n_items, embedding_dim)

                # MLP
                self.mlp_user_embedding = nn.Embedding(n_users, embedding_dim)
                self.mlp_item_embedding = nn.Embedding(n_items, embedding_dim)
                self.mlp_fc1 = nn.Linear(2 * embedding_dim, 16)
                self.mlp_fc2 = nn.Linear(16, 8)
                
                # Content-based
                self.content_based = nn.Linear(24, 8)

                # Merge
                self.fc1 = nn.Linear(embedding_dim + 8 + 8, 16)
                self.fc2 = nn.Linear(16, 1)
                
            # [5] GMF + MLP + Content-based #3
            if model_id == 5:
                # GMF
                self.gmf_user_embedding = nn.Embedding(n_users, embedding_dim)
                self.gmf_item_embedding = nn.Embedding(n_items, embedding_dim)

                # MLP
                self.mlp_user_embedding = nn.Embedding(n_users, embedding_dim)
                self.mlp_item_embedding = nn.Embedding(n_items, embedding_dim)
                self.mlp_fc1 = nn.Linear(2 * embedding_dim, 16)
                self.mlp_fc2 = nn.Linear(16, 8)
                
                # Content-based (dot product)
                self.content_based = nn.Linear(24, 8)
                
                # Content-based (separately)
                self.content_based_sep = nn.Linear(2 * 24, 8)
                
                # Merge
                self.fc1 = nn.Linear(embedding_dim + 8 + 8 + 8, 16)
                self.fc2 = nn.Linear(16, 1)
                
    def forward(self, x):
        user_ids = x[:, 0]
        item_ids = x[:, 1]
        
        user_embedding = self.user_embedding(user_ids)
        item_embedding = self.item_embedding(item_ids)
        
        # --- Only Embeddings Models --- #
        if not self.hybrid:
            
            # [1] One fully-connected layer without bias
            if self.model_id == 1:
                x = self.fc(user_embedding * item_embedding)
                x = torch.sigmoid(x)

            # [2] Two fully-connected layers without bias
            if self.model_id == 2:
                x = torch.relu(self.fc1(user_embedding * item_embedding))
                x = self.fc2(x)
                x = torch.sigmoid(x)

            # [3] Three fully-connected layers without bias
            if self.model_id == 3:
                x = torch.relu(self.fc1(user_embedding * item_embedding))
                x = torch.relu(self.fc2(x))
                x = self.fc3(x)
                x = torch.sigmoid(x)

            # [4] Four fully-connected layers without bias
            if self.model_id == 4:
                x = torch.relu(self.fc1(user_embedding * item_embedding))
                x = torch.relu(self.fc2(x))
                x = torch.relu(self.fc3(x))
                x = self.fc4(x)
                x = torch.sigmoid(x)

            # [5] Five fully-connected layers without bias
            if self.model_id == 5:
                x = torch.relu(self.fc1(user_embedding * item_embedding))
                x = torch.relu(self.fc2(x))
                x = torch.relu(self.fc3(x))
                x = torch.relu(self.fc4(x))
                x = self.fc5(x)
                x = torch.sigmoid(x)

            # [6] Six fully-connected layers without bias
            if self.model_id == 6:
                x = torch.relu(self.fc1(user_embedding * item_embedding))
                x = torch.relu(self.fc2(x))
                x = torch.relu(self.fc3(x))
                x = torch.relu(self.fc4(x))
                x = torch.relu(self.fc5(x))
                x = self.fc6(x)
                x = torch.sigmoid(x)

            # [7] GMF + MLP
            if self.model_id == 7:
                # GMF
                gmf_user_embedding = self.gmf_user_embedding(user_ids)
                gmf_item_embedding = self.gmf_item_embedding(item_ids)
                gmf_x = gmf_user_embedding * gmf_item_embedding

                # MLP
                mlp_user_embedding = self.mlp_user_embedding(user_ids)
                mlp_item_embedding = self.mlp_item_embedding(item_ids)
                mlp_x = torch.cat([mlp_user_embedding, mlp_item_embedding], dim=1)
                mlp_x = torch.relu(self.mlp_fc1(mlp_x))
                mlp_x = torch.relu(self.mlp_fc2(mlp_x))

                # Contact results from diffrent approaches
                x = torch.cat([gmf_x, mlp_x], dim=1)
                
                x = torch.sigmoid(self.fc(x))
        
        # --- Hybrid Models --- #
        if self.hybrid:
            content_based_features_dot_prod = x[:, 2:26].to(torch.float32)
            content_based_separately = x[:, 26:].to(torch.float32)
            
            # [1] One fully-connected layer without bias
            if self.model_id == 1:
                embeddings = user_embedding * item_embedding
                x = torch.cat([embeddings, content_based_features_dot_prod], dim=1)
                x = torch.sigmoid(self.fc(x))
                
            # [2] Three fully-connected layer with bias
            if self.model_id == 2:
                embeddings = torch.relu(self.embedding(user_embedding * item_embedding))
                
                content_based = torch.relu(self.fc_content_1(content_based_features_dot_prod))
                content_based = torch.relu(self.fc_content_2(content_based))
                content_based = torch.relu(self.fc_content_3(content_based))
                
                x = torch.cat([embeddings, content_based], dim=1)
                x = self.fc(x)
                x = torch.sigmoid(x)
                
            # [3] GMF + MLP + Content-based #1
            if self.model_id == 3:
                # GMF
                gmf_user_embedding = self.gmf_user_embedding(user_ids)
                gmf_item_embedding = self.gmf_item_embedding(item_ids)
                gmf_x = gmf_user_embedding * gmf_item_embedding

                # MLP
                mlp_user_embedding = self.mlp_user_embedding(user_ids)
                mlp_item_embedding = self.mlp_item_embedding(item_ids)
                mlp_x = torch.cat([mlp_user_embedding, mlp_item_embedding], dim=1)
                mlp_x = torch.relu(self.mlp_fc1(mlp_x))
                mlp_x = torch.relu(self.mlp_fc2(mlp_x))
                
                # Content-based (dot product)
                content_based = self.content_based(content_based_features_dot_prod)
                
                # Content based (separately)
                content_based_sep = torch.relu(self.cb_fc1(content_based_separately))
                content_based_sep = torch.relu(self.cb_fc2(content_based_sep))
                
                # Contact results from diffrent approaches
                x = torch.cat([gmf_x, mlp_x, content_based, content_based_sep], dim=1)
                x = torch.sigmoid(self.fc2(torch.relu(self.fc1(x))))
        
            # [3] GMF + MLP + Content-based #2
            if self.model_id == 4:
                 # GMF
                gmf_user_embedding = self.gmf_user_embedding(user_ids)
                gmf_item_embedding = self.gmf_item_embedding(item_ids)
                gmf_x = gmf_user_embedding * gmf_item_embedding

                # MLP
                mlp_user_embedding = self.mlp_user_embedding(user_ids)
                mlp_item_embedding = self.mlp_item_embedding(item_ids)
                mlp_x = torch.cat([mlp_user_embedding, mlp_item_embedding], dim=1)
                mlp_x = torch.relu(self.mlp_fc1(mlp_x))
                mlp_x = torch.relu(self.mlp_fc2(mlp_x))
                
                # Content-based (dot product)
                content_based = self.content_based(content_based_features_dot_prod)
                
                # Contact results from diffrent approaches
                x = torch.cat([gmf_x, mlp_x, content_based], dim=1)
                
                x = torch.sigmoid(self.fc2(torch.relu(self.fc1(x))))
                
            # [5] GMF + MLP + Content-based #3
            if self.model_id == 5:
                # GMF
                gmf_user_embedding = self.gmf_user_embedding(user_ids)
                gmf_item_embedding = self.gmf_item_embedding(item_ids)
                gmf_x = gmf_user_embedding * gmf_item_embedding

                # MLP
                mlp_user_embedding = self.mlp_user_embedding(user_ids)
                mlp_item_embedding = self.mlp_item_embedding(item_ids)
                mlp_x = torch.cat([mlp_user_embedding, mlp_item_embedding], dim=1)
                mlp_x = torch.relu(self.mlp_fc1(mlp_x))
                mlp_x = torch.relu(self.mlp_fc2(mlp_x))
                
                # Content-based (dot product)
                content_based = self.content_based(content_based_features_dot_prod)
                
                # Content based (separately)
                content_based_sep = self.content_based_sep(content_based_separately)
                
                # Contact results from diffrent approaches
                x = torch.cat([gmf_x, mlp_x, content_based, content_based_sep], dim=1)
                
                x = torch.sigmoid(self.fc2(torch.relu(self.fc1(x))))
        
        return x


# Quick test of the recommender

In [7]:
# Prepare items_df
items_df = interactions_df.loc[:, ['item_id'] + base_item_features].drop_duplicates()

In [8]:
# Fit method
params = {
    'batch_size': 512, 
    'embedding_dim': 6,
    'lr': 0.01,
    'n_epochs': 1.0, 
    'n_neg_per_pos': 1.0,
    'weight_decay': 0.01,
    'model_id': 5,
    "hybrid": True
}

nn_recommender = NNRecommender(**params)
nn_recommender.fit(interactions_df, None, None)

In [9]:
# Recommender method

recommendations = nn_recommender.recommend(pd.DataFrame([[1], [2], [3], [4], [5]], columns=['user_id']), items_df, 10)

recommendations = pd.merge(recommendations, items_df, on='item_id', how='left')
display(HTML(recommendations.to_html()))

,user_id,item_id,score,term,length_of_stay_bucket,rate_plan,room_segment,n_people_bucket,weekend_stay
0,1,22,0.999242,OffSeason,[2-3],Standard,[160-260],[3-4],True
1,1,51,0.998737,OffSeason,[2-3],Nonref,[160-260],[3-4],True
2,1,34,0.998315,OffSeason,[2-3],Standard,[160-260],[3-4],False
3,1,57,0.998115,OffSeason,[2-3],Nonref,[160-260],[3-4],False
4,1,199,0.997876,LowSeason,[2-3],Nonref,[160-260],[3-4],True
5,1,55,0.997813,OffSeason,[2-3],Nonref,[160-260],[2-2],True
6,1,78,0.997510,HighSeason,[2-3],Nonref,[160-260],[3-4],True
7,1,142,0.997408,LowSeason,[2-3],Nonref,[160-260],[2-2],True
8,1,9,0.997212,HighSeason,[2-3],Standard,[160-260],[3-4],True
9,1,159,0.997120,LowSeason,[2-3],Standard,[160-260],[3-4],True


# Tuning method

In [8]:
from evaluation_and_testing.testing import evaluate_train_test_split_implicit

seed = 6789

In [9]:
from hyperopt import hp, fmin, tpe, Trials, space_eval
import traceback

def tune_recommender(recommender_class, interactions_df, items_df, 
                     param_space, max_evals=1, show_progressbar=True, seed=6789):
    
    # Tune method
    def loss(tuned_params):
        
        # Create recommender class with tuned params
        recommender = recommender_class(seed=seed, **tuned_params)
        
        # Print tuned params
        print(f"Tuned params = {tuned_params}")
        
        # Evaluate recommender on whole interactions_df
        hr1, hr3, hr5, hr10, ndcg1, ndcg3, ndcg5, ndcg10 = evaluate_train_test_split_implicit(
            recommender, interactions_df, items_df, seed=seed)
        
        # Print @HR10 result
        print(f"@HR10 = {hr10}")
        
        return -hr10

    # Tune
    n_tries = 1
    succeded = False
    try_id = 0
    while not succeded and try_id < n_tries:
        try:
            trials = Trials()
            best_param_set = fmin(loss, space=param_space, algo=tpe.suggest, 
                                  max_evals=max_evals, show_progressbar=show_progressbar, trials=trials, verbose=True)
            succeded = True
        except:
            traceback.print_exc()
            try_id += 1
            
    if not succeded:
        return None
    
    # Convert indexes from hp.choice to real data
    params = space_eval(param_space, best_param_set)
    
    return params

## Tuning of the recommender

<span style="color:red"><font size="4">**Task:**</font></span><br> 
Tune your model using the code below. You only need to put the class name of your recommender and choose an appropriate parameter space.

In [42]:
# --- This is only example! --- #
# All the results of the tuning performed for the different architectures can be found in .xls files

# Define param space
param_space = {
    'n_neg_per_pos': hp.quniform('n_neg_per_pos', 1, 10, 1),
    'weight_decay': hp.uniform('weight_decay', 0.001, 0.03),
    "n_epochs": hp.quniform('n_epochs', 1, 20, 2),
    "batch_size": hp.choice('batch_size', [int(2 ** i) for i in range(6, 11)]),
    "lr": hp.uniform('lr', 0.001, 0.03),
    "embedding_dim": hp.choice('embedding_dim', [3, 4, 6, 8, 10, 12, 16, 20, 24, 32, 48, 56, 64]),
    "model_id": 5,
    "hybrid": True
}

# Tune recommender based on param space
best_param_set = tune_recommender(NNRecommender, interactions_df, items_df,
                                  param_space, max_evals=5, show_progressbar=True, seed=seed)

print("Best parameters:")
print(best_param_set)

Tuned params = {'batch_size': 512, 'embedding_dim': 10, 'hybrid': True, 'lr': 0.02599983772536897, 'model_id': 5, 'n_epochs': 16.0, 'n_neg_per_pos': 9.0, 'weight_decay': 0.009131690473102892}
@HR10 = 0.22131704005431094                          
Tuned params = {'batch_size': 64, 'embedding_dim': 10, 'hybrid': True, 'lr': 0.0086557402438012, 'model_id': 5, 'n_epochs': 20.0, 'n_neg_per_pos': 3.0, 'weight_decay': 0.023225499586781505}
@HR10 = 0.1581805838424983                                                        
Tuned params = {'batch_size': 256, 'embedding_dim': 48, 'hybrid': True, 'lr': 0.021941059864547616, 'model_id': 5, 'n_epochs': 18.0, 'n_neg_per_pos': 10.0, 'weight_decay': 0.019359871402405863}
@HR10 = 0.1826205023761032                                                        
Tuned params = {'batch_size': 256, 'embedding_dim': 4, 'hybrid': True, 'lr': 0.01978351140194205, 'model_id': 5, 'n_epochs': 6.0, 'n_neg_per_pos': 9.0, 'weight_decay': 0.0033440056736203905}
@HR10 = 0.181

# Final evaluation

<span style="color:red"><font size="4">**Task:**</font></span><br> 
Run the final evaluation of your recommender and present its results against the Amazon and Netflix recommenders' results. You just need to give the class name of your recommender and its tuned parameters below.

It's optional, but for better effect you can include here the results from all recommenders created during in this class.

In [13]:
# Best Models

# --- Neural Network Architecture [1] Embeddings --- #
params = {
    'model_id': 1,
    'hybrid': False, 
    'n_neg_per_pos': 10.0,
    'n_epochs': 4.0,
    'batch_size': 256, 
    'embedding_dim': 32,
    'lr': 0.0093946405726787906,
    'weight_decay': 0.002822552453575869
}

NNRecommender1 = NNRecommender(**params)

# --- Neural Network Architecture [7] Embeddings --- #
params = {
    'model_id': 7,
    'hybrid': False, 
    'n_neg_per_pos': 3.0,
    'n_epochs': 46.0,
    'batch_size': 64, 
    'embedding_dim': 48,
    'lr': 0.01611580698017756,
    'weight_decay': 0.015768096050434624
}

NNRecommender7 = NNRecommender(**params)

# --- Neural Network Architecture [3] Embeddings + Content-based (Hybrid) --- #
params = {
    'model_id': 3,
    'hybrid': True, 
    'n_neg_per_pos': 5.0,
    'n_epochs': 34.0,
    'batch_size': 256,
    'embedding_dim': 24,
    'lr': 0.024226437162031808,
    'weight_decay': 0.01800282239612605
}

NNRecommender3Hybrid = NNRecommender(**params)

# --- Neural Network Architecture [4] Embeddings + Content-based (Hybrid) --- #
params = {
    'model_id': 4, 
    'hybrid': True, 
    'n_neg_per_pos': 9.0, 
    'n_epochs': 37.0, 
    'batch_size': 1024,
    'embedding_dim': 56, 
    'lr': 0.00484012928805184, 
    'weight_decay': 0.0016429769911333152
}

NNRecommender4Hybrid = NNRecommender(**params)

# --- Neural Network Architecture [5] Embeddings + Content-based (Hybrid) --- #
params = {
    'model_id': 5, 
    'hybrid': True, 
    'n_neg_per_pos': 7.0, 
    'n_epochs': 2.0,
    'batch_size': 512,
    'embedding_dim': 8, 
    'lr': 0.011345327404520558, 
    'weight_decay': 0.011422233277123908
}

NNRecommender5Hybrid = NNRecommender(**params)

# --- Content Based Recommenders form previous project --- #
from recommenders.content_based_recommenders import LinearRegressionCBUIRecommender, RandomForestCBUIRecommender, CatBoostRegressorCBUIRecommender, XGBoostCBUIRecommender

# LinearRegressionCBUIRecommender
params = {
    'n_neg_per_pos': 1.0
}

linearRegressionCBUIRecommender = LinearRegressionCBUIRecommender(**params)
# RandomForestCBUIRecommender
params = {
    'max_depth': 7.0, 
    'min_samples_split': 7.0,
    'n_estimators': 172.0, 
    'n_neg_per_pos': 8.0
}

randomForestCBUIRecommender = RandomForestCBUIRecommender(**params)

# XGBoostCBUIRecommender
params = {
    'learning_rate': 0.013782403981999257, 
    'max_depth': 3.0, 
    'min_samples_split': 4.0, 
    'n_estimators': 197.0, 
    'n_neg_per_pos': 9.0
}

xGBoostCBUIRecommender = XGBoostCBUIRecommender(**params)
# CatBoostRegressorCBUIRecommender
params = {
    'iterations': 600, 
    'learning_rate': 0.02, 
    'depth': 6, 
    'l2_leaf_reg': 3.0,
    'n_neg_per_pos': 10.0
}

catBoostRegressorCBUIRecommender = CatBoostRegressorCBUIRecommender(**params)

# Amazon Recommender
from recommenders.amazon_recommender import AmazonRecommender

AmazonRecommender = AmazonRecommender()

# List of recommenders
recommenders = {
    "NNRecommender1": NNRecommender1,
    "NNRecommender7": NNRecommender7,
    "NNRecommender3Hybrid": NNRecommender3Hybrid,
    "NNRecommender4Hybrid": NNRecommender4Hybrid,
    "NNRecommender5Hybrid": NNRecommender5Hybrid,
    "LinearRegressionCBUIRecommender": linearRegressionCBUIRecommender,
    "RandomForestCBUIRecommender": randomForestCBUIRecommender,
    "XGBoostCBUIRecommender": xGBoostCBUIRecommender,
    "CatBoostRegressorCBUIRecommender": catBoostRegressorCBUIRecommender,
    "AmazonRecommender": AmazonRecommender
}

# Prepare list for all results
all_results = []

# Evaluate recommenders
for name, recommender in recommenders.items():
    result = [[name] + list(evaluate_train_test_split_implicit(recommender, interactions_df, items_df))]
    result = pd.DataFrame(result, columns=['Recommender', 'HR@1', 'HR@3', 'HR@5', 'HR@10', 'NDCG@1', 'NDCG@3', 'NDCG@5', 'NDCG@10'])
    all_results.append(result)
    
# Concatenate all recommenders result into one DataFrame
all_results = pd.concat(all_results).reset_index(drop=True)

# Display evaluation results
display(all_results)

,Recommender,HR@1,HR@3,HR@5,HR@10,NDCG@1,NDCG@3,NDCG@5,NDCG@10
0,NNRecommender1,0.043109,0.129328,0.176511,0.248812,0.043109,0.092663,0.112359,0.135561
1,NNRecommender7,0.042770,0.125594,0.172437,0.240665,0.042770,0.090671,0.110428,0.132312
2,NNRecommender3Hybrid,0.047183,0.128310,0.187373,0.255601,0.047183,0.093924,0.118632,0.140749
3,NNRecommender4Hybrid,0.058045,0.143585,0.190767,0.265105,0.058045,0.106859,0.126495,0.150397
4,NNRecommender5Hybrid,0.057366,0.138493,0.165309,0.262729,0.057366,0.103796,0.114795,0.146049
5,LinearRegressionCBUIRecommender,0.058724,0.142227,0.184997,0.243720,0.058724,0.105453,0.123352,0.142156
6,RandomForestCBUIRecommender,0.056348,0.104888,0.154786,0.233876,0.056348,0.085729,0.106877,0.131733
7,XGBoostCBUIRecommender,0.058045,0.142227,0.187373,0.252206,0.058045,0.105380,0.124273,0.145224
8,CatBoostRegressorCBUIRecommender,0.046843,0.110319,0.165648,0.244060,0.046843,0.085070,0.107084,0.132890
9,AmazonRecommender,0.044128,0.118805,0.160557,0.223693,0.044128,0.086755,0.104216,0.124468


# Summary

<span style="color:red"><font size="4">**Task:**</font></span><br> 
Write a summary of your experiments. What worked well and what did not? What are your thoughts how could you possibly further improve the model?